<p style="padding: 10px; border: 1px solid black;">
<img src="./utils/MLU-NEW-logo.png" alt="drawing" width="400"/> <br/>

# <a name="0"> Image Classification using AutoGluon </a>

In this notebook, we will load images and the corresponding labels into [AutoGluon](https://autogluon.mxnet.io/index.html) and use this data to obtain a neural network that can classify new images. This is different from traditional machine learning where we need to manually define the neural network and then specify the hyperparameters in the training process. Instead, with just a single call to AutoGluon’s fit function, AutoGluon automatically trains many models with different hyperparameter configurations and returns the model that achieved the highest level of accuracy.
    
1. <a href="#Download-the-Image-Dataset">Download the Image Dataset</a>
2. <a href="#AutoGluon-ImagePredictor">AutoGluon ImagePredictor</a>
3. <a href="#AutoGluon-Training-Results">AutoGluon Training Results</a>
4. <a href="#Generate-image-features-with-a-classifier">Generate image features with a classifier</a>
5. <a href="#Making-Predictions-with-ImagePredictor">Making Predictions with ImagePredictor</a>
6. <a href="#Search-Space-and-Hyperparameter-Optimization">Search Space and Hyperparameter Optimization</a>

Note: Please use **GPU** for training. CPU training will lead to an unceasing running script. 

In [1]:
# Install AutoGluon
# ! pip install -q autogluon

Let's import the ImagePredictor

In [2]:
from autogluon.vision import ImagePredictor
import autogluon.core as ag
import pandas as pd

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.7.0` and `torch==1.10.1+cu102` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '


To use AutoGluon for computer vision task training, we need to organize our data with the following structure:

    data/
    ├── train/
        ├── class1/
        ├── class2/
        ├── class3/
        ├── ...
    ├── test/
        ├── class1/
        ├── class2/
        ├── class3/
        ├── ...

Here each subfolder contains all images that belong to that category, e.g., `class1` contains all images belonging to the first class. We generally recommend at least 100 training images per class for reasonable classification performance, but this might depend on the type of images in your specific use-case.

## <a id="Download-the-Image-Dataset">Download the Image Dataset</a>
For demonstration purposes, we use a subset of the __Shopee-IET__ dataset from [Kaggle](https://www.kaggle.com/competitions). Each image in this data depicts a clothing item and the corresponding label specifies its clothing category. Our subset of the data contains the following possible labels: BabyPants, BabyShirt, womencasualshoes, womenchiffontop.

We download the data subset and create training/test dataset folders like below. If you use this on your own dataset, just point it to your training or test folder. Example: `train_dataset = ImagePredictor.Dataset.from_folder('mydataset/train')`

(<a href="#0">Go to top</a>)

In [3]:
path = 'https://autogluon.s3.amazonaws.com/datasets/shopee-iet.zip'
df_train, _, df_test = ImagePredictor.Dataset.from_folders(path)

data/
├── test/
└── train/


Let's print the training dataset.

In [4]:
print(df_train)

                                                 image  label
0    /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      0
1    /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      0
2    /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      0
3    /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      0
4    /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      0
..                                                 ...    ...
795  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      3
796  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      3
797  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      3
798  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      3
799  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      3

[800 rows x 2 columns]


##  <a id="AutoGluon-ImagePredictor">AutoGluon ImagePredictor</a>

Now, let's fit a __classifier__ using AutoGluon [predictor.fit()](https://auto.gluon.ai/stable/tutorials/image_prediction/beginner.html). Within fit, the dataset is __automatically__ split into training and validation sets. The model with the best hyperparameter configuration is selected based on its performance on the __validation set__.

(<a href="#0">Go to top</a>)

In [5]:
predictor = ImagePredictor(path="AutogluonModels/Image")

time_limit = 10 * 60 # how long fit() should run (in seconds)
predictor.fit(df_train,
              #epochs=10,
              time_limit=time_limit,
              ngpus_per_trial=1
             )

INFO:gluoncv.auto.tasks.image_classification:Randomly split train_data into train[716]/validation[84] splits.
INFO:gluoncv.auto.tasks.image_classification:Starting fit without HPO
INFO:ImageClassificationEstimator:modified configs(<old> != <new>): {
INFO:ImageClassificationEstimator:root.img_cls.model   resnet50_v1 != resnet50_v1b
INFO:ImageClassificationEstimator:root.valid.batch_size 128 != 16
INFO:ImageClassificationEstimator:root.train.lr        0.1 != 0.01
INFO:ImageClassificationEstimator:root.train.data_dir  ~/.mxnet/datasets/imagenet != auto
INFO:ImageClassificationEstimator:root.train.rec_train ~/.mxnet/datasets/imagenet/rec/train.rec != auto
INFO:ImageClassificationEstimator:root.train.batch_size 128 != 16
INFO:ImageClassificationEstimator:root.train.rec_train_idx ~/.mxnet/datasets/imagenet/rec/train.idx != auto
INFO:ImageClassificationEstimator:root.train.num_training_samples 1281167 != -1
INFO:ImageClassificationEstimator:root.train.epochs    10 != 15
INFO:ImageClassificati

In [6]:
predictor.save("AutogluonModels/Image/ImagePredictor.ag")

##  <a id="AutoGluon-Training-Results">AutoGluon Training Results</a>
 
Autogluon also provides the training results, which can be accessed by calling `predictor.fit_summary()`. 

(<a href="#0">Go to top</a>)

In [7]:
fit_result = predictor.fit_summary()

In [8]:
fit_result

{'train_acc': 0.4744318181818182,
 'valid_acc': 0.75,
 'total_time': 608.5926694869995,
 'best_config': {'model': 'resnet50_v1b',
  'lr': 0.01,
  'num_trials': 1,
  'epochs': 15,
  'batch_size': 16,
  'nthreads_per_trial': 128,
  'ngpus_per_trial': 1,
  'time_limits': 600,
  'search_strategy': 'random',
  'dist_ip_addrs': None,
  'log_dir': 'AutogluonModels/Image/07767bde',
  'num_workers': 4,
  'gpus': [0],
  'seed': 647,
  'final_fit': False,
  'estimator': gluoncv.auto.estimators.image_classification.image_classification.ImageClassificationEstimator,
  'wall_clock_tick': 1659562789.3028562,
  'problem_type': 'multiclass'},
 'fit_history': {'train_acc': 0.4744318181818182,
  'valid_acc': 0.75,
  'total_time': 608.5926694869995,
  'best_config': {'model': 'resnet50_v1b',
   'lr': 0.01,
   'num_trials': 1,
   'epochs': 15,
   'batch_size': 16,
   'nthreads_per_trial': 128,
   'ngpus_per_trial': 1,
   'time_limits': 600,
   'search_strategy': 'random',
   'dist_ip_addrs': None,
   'log_

We can access certain results from this summary. For example, training and validation accuracies below.

In [9]:
print('Train acc: %.3f, val acc: %.3f' %(fit_result['train_acc'], fit_result['valid_acc']))

Train acc: 0.474, val acc: 0.750


The best model and optimum hyperparameters: Learning rate, batch size, epochs can be printed with this:

In [10]:
fit_result['fit_history']['best_config']

{'model': 'resnet50_v1b',
 'lr': 0.01,
 'num_trials': 1,
 'epochs': 15,
 'batch_size': 16,
 'nthreads_per_trial': 128,
 'ngpus_per_trial': 1,
 'time_limits': 600,
 'search_strategy': 'random',
 'dist_ip_addrs': None,
 'log_dir': 'AutogluonModels/Image/07767bde',
 'num_workers': 4,
 'gpus': [0],
 'seed': 647,
 'final_fit': False,
 'estimator': gluoncv.auto.estimators.image_classification.image_classification.ImageClassificationEstimator,
 'wall_clock_tick': 1659562789.3028562,
 'problem_type': 'multiclass'}

##  <a id="Generate-image-features-with-a-classifier">Generate image features with a classifier</a>

Extracting representation from the whole image learned by a model is also very useful. The `predict_feature` function returns a vector of image features.

(<a href="#0">Go to top</a>)

In [11]:
image_path = df_test.iloc[0]['image']
feature = predictor.predict_feature(image_path)
print(feature)

                                       image_feature
0  [0.3753202, 0.0, 0.31749323, 1.2149992, 0.6535...


##  <a id="Making-Predictions-with-ImagePredictor">Making Predictions with ImagePredictor</a>

(<a href="#0">Go to top</a>)

We can call the predict function to run on different images.

In [12]:
image_path = df_test.iloc[0]['image']
predictor.predict(image_path)

,class,score,id
0,BabyShirt,0.488706,1


Let's get predictions on the test set.

In [13]:
pred = predictor.predict(df_test)
print(pred)

               class     score  id  \
0          BabyShirt  0.488706   1   
1          BabyShirt  0.887769   1   
2          BabyShirt  0.432509   1   
3   womencasualshoes  0.593805   2   
4          BabyShirt  0.790615   1   
..               ...       ...  ..   
75   womenchiffontop  0.733472   3   
76   womenchiffontop  0.360697   3   
77   womenchiffontop  0.780986   3   
78         BabyPants  0.484973   0   
79   womenchiffontop  0.742945   3   

                                                image  
0   /home/ec2-user/.gluoncv/datasets/shopee-iet/da...  
1   /home/ec2-user/.gluoncv/datasets/shopee-iet/da...  
2   /home/ec2-user/.gluoncv/datasets/shopee-iet/da...  
3   /home/ec2-user/.gluoncv/datasets/shopee-iet/da...  
4   /home/ec2-user/.gluoncv/datasets/shopee-iet/da...  
..                                                ...  
75  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...  
76  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...  
77  /home/ec2-user/.gluoncv/datasets/sh

In [14]:
# Evaluate image predictor on test data rather than retrieving predictions
predictor.evaluate(df_test)

(0.675, 1.0)

## <a id="Search-Space-and-Hyperparameter-Optimization">Search Space and Hyperparameter Optimization</a>

This section dives into a few options for advanced control over ImagePredictor's fitting process such as specifying which:

- Networks to Try
- Training Hyperparameters
- Search Algorithms

(<a href="#0">Go to top</a>)

### Specify which Networks to Try
We start with specifying the pretrained neural network candidates. Given such a list, AutoGluon tries to train different networks from this list to identify the best-performing candidate.

In [15]:
# you may choose more than 70+ available model in the model zoo provided by GluonCV:
model_list = ImagePredictor.list_models()
model_list

('resnet18_v1',
 'resnet34_v1',
 'resnet50_v1',
 'resnet101_v1',
 'resnet152_v1',
 'resnet18_v2',
 'resnet34_v2',
 'resnet50_v2',
 'resnet101_v2',
 'resnet152_v2',
 'resnest14',
 'resnest26',
 'resnest50',
 'resnest101',
 'resnest200',
 'resnest269',
 'se_resnet18_v1',
 'se_resnet34_v1',
 'se_resnet50_v1',
 'se_resnet101_v1',
 'se_resnet152_v1',
 'se_resnet18_v2',
 'se_resnet34_v2',
 'se_resnet50_v2',
 'se_resnet101_v2',
 'se_resnet152_v2',
 'vgg11',
 'vgg13',
 'vgg16',
 'vgg19',
 'vgg11_bn',
 'vgg13_bn',
 'vgg16_bn',
 'vgg19_bn',
 'alexnet',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'squeezenet1.0',
 'squeezenet1.1',
 'googlenet',
 'inceptionv3',
 'xception',
 'xception71',
 'mobilenet1.0',
 'mobilenet0.75',
 'mobilenet0.5',
 'mobilenet0.25',
 'mobilenetv2_1.0',
 'mobilenetv2_0.75',
 'mobilenetv2_0.5',
 'mobilenetv2_0.25',
 'mobilenetv3_large',
 'mobilenetv3_small',
 'cifar_resnet20_v1',
 'cifar_resnet56_v1',
 'cifar_resnet110_v1',
 'cifar_resnet20_v2',
 'cifar_

In [16]:
# Provide a list of networks for AutoGluon to try
model = ag.Categorical('resnet18_v1b', 'mobilenetv3_small')

### Specify the Training Hyperparameters
We can manually specify many crucial hyper-parameters, with specific value or search space

In [17]:
# Specify the batch size value and learning rate search space 
batch_size = 8
lr = ag.Categorical(1e-2, 1e-3)

### Specify the Search Algorithms
Beyond simply specifying the space of hyperparameter configurations to search over, you can also tell AutoGluon what strategy it should employ to actually search through this space. This process of finding good hyperparameters from a given search space is commonly referred to as hyperparameter optimization (HPO) or hyperparameter tuning

We would be using random search for HPO.

In [19]:
hyperparameters={'model': model, 'batch_size': batch_size, 'lr': lr, 'epochs': 2}
predictor = ImagePredictor()
predictor.fit(df_train, time_limit=10*60, hyperparameters=hyperparameters)
print('Top-1 val acc: %.3f' % predictor.fit_summary()['valid_acc'])


INFO:gluoncv.auto.tasks.image_classification:Randomly split train_data into train[731]/validation[69] splits.
INFO:gluoncv.auto.tasks.image_classification:Starting fit without HPO
INFO:ImageClassificationEstimator:modified configs(<old> != <new>): {
INFO:ImageClassificationEstimator:root.img_cls.model   resnet50_v1 != resnet18_v1b
INFO:ImageClassificationEstimator:root.valid.batch_size 128 != 8
INFO:ImageClassificationEstimator:root.train.lr        0.1 != 0.01
INFO:ImageClassificationEstimator:root.train.data_dir  ~/.mxnet/datasets/imagenet != auto
INFO:ImageClassificationEstimator:root.train.rec_train ~/.mxnet/datasets/imagenet/rec/train.rec != auto
INFO:ImageClassificationEstimator:root.train.batch_size 128 != 8
INFO:ImageClassificationEstimator:root.train.rec_train_idx ~/.mxnet/datasets/imagenet/rec/train.idx != auto
INFO:ImageClassificationEstimator:root.train.num_training_samples 1281167 != -1
INFO:ImageClassificationEstimator:root.train.epochs    10 != 2
INFO:ImageClassificationE

42433KB [00:01, 36670.24KB/s]                           
INFO:ImageClassificationEstimator:Start training from [Epoch 0]
INFO:ImageClassificationEstimator:Epoch[0] Batch [49]	Speed: 4.921916 samples/sec	accuracy=0.422500	lr=0.010000
INFO:ImageClassificationEstimator:[Epoch 0] training: accuracy=0.464286
INFO:ImageClassificationEstimator:[Epoch 0] speed: 4 samples/sec	time cost: 146.568436
INFO:ImageClassificationEstimator:[Epoch 0] validation: top1=0.695652 top5=1.000000
INFO:ImageClassificationEstimator:[Epoch 0] Current best top-1: 0.695652 vs previous -inf, saved to /home/ec2-user/SageMaker/MLU-WKSP-Tuning-Autogluon/notebooks/0e651630/.trial_0/best_checkpoint.pkl
INFO:ImageClassificationEstimator:Epoch[1] Batch [49]	Speed: 4.909809 samples/sec	accuracy=0.605000	lr=0.010000
INFO:ImageClassificationEstimator:[Epoch 1] training: accuracy=0.627747
INFO:ImageClassificationEstimator:[Epoch 1] speed: 4 samples/sec	time cost: 148.307698
INFO:ImageClassificationEstimator:[Epoch 1] validation

Top-1 val acc: 0.768


<p style="padding: 10px; border: 1px solid black;">
<img src="./utils/MLU-NEW-logo.png" alt="drawing" width="400"/> <br/>

# Thank you!